In [16]:
%pip install -U "anyio>=4.2" > /dev/null

In [1]:
%pip install opentrons

In [1]:
%pip install prefect

In [ ]:
!git clone https://github.com/AccelerationConsortium/ac-training-lab.git

## Prefect Cloud Login Required

Before running the cells below, please log in to Prefect Cloud.

The following parameters should be the same as those used in the **ot2_orchestrator_prefect notebook**.

### How to get your API Key:
1. Go to [Prefect Cloud](https://app.prefect.cloud)
2. Click the workspace name in the top-left (e.g., `default`)
3. Select **API Keys** from the menu
4. Click the + icon to create a new key
5. Copy the key and paste it into the input box below

### How to get your API URL:
1. Open your workspace
2. Copy the URL from your browser — it looks like:
https://app.prefect.cloud/account/.../workspace/...

3. Then convert it to the following format (keep the content after the colons unchanged): https://api.prefect.cloud/api/accounts/.../workspaces/...

(e.g. https://api.prefect.cloud/api/accounts/12345678-abcd-1234-abcd-1234567890ab/workspaces/abcdef12-3456-7890-abcd-abcdef123456)

In [6]:
# @title Prefect Cloud Authentication
import os
from prefect.deployments import run_deployment

PREFECT_API_KEY = "" # @param {type:"string"}
PREFECT_API_URL = "https://api.prefect.cloud/api/accounts/.../workspaces/..." # @param {type:"string"}

os.environ["PREFECT_API_KEY"] = PREFECT_API_KEY
os.environ["PREFECT_API_URL"] = PREFECT_API_URL

!prefect deployment ls

                                  Deployments                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Name                          ┃ ID                           ┃ Work Pool     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mix-color/mix-color           │ dc972dfb-2cdc-411b-8096-ec1… │               │
│ move-sensor-back/move-sensor… │ 9835a1f2-6b19-4837-86fa-fcb… │               │
│ move-sensor-to-measurement-p… │ ad982977-d6fb-412a-b14b-513… │               │
└───────────────────────────────┴──────────────────────────────┴───────────────┘


In [4]:
%%writefile device.py
import json
import opentrons.simulate
from prefect import flow, serve
import os

# ------------------- OT-2 Setup -------------------
protocol = opentrons.simulate.get_protocol_api("2.12")
protocol.home()

# Load Labware
with open("/content/ac-training-lab/src/ac_training_lab/ot-2/_scripts/ac_color_sensor_charging_port.json", encoding="utf-8") as f1:
    labware_def1 = json.load(f1)
    tiprack_2 = protocol.load_labware_from_definition(labware_def1, 10)

with open("/content/ac-training-lab/src/ac_training_lab/ot-2/_scripts/ac_6_tuberack_15000ul.json", encoding="utf-8") as f2:
    labware_def2 = json.load(f2)
    reservoir = protocol.load_labware_from_definition(labware_def2, 3)

plate = protocol.load_labware("corning_96_wellplate_360ul_flat", location=1)
tiprack_1 = protocol.load_labware("opentrons_96_tiprack_300ul", location=9)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2", mount="right", tip_racks=[tiprack_1]
)
p300.well_bottom_clearance.dispense = 8

print("Labwares loaded")

# ------------------- Prefect Tasks -------------------
@flow
def mix_color(R, Y, B, mix_well):
    total = R + Y + B
    if total > 300:
        raise ValueError("The sum of the proportions must be <= 300")

    position = ["B1", "B2", "B3"]
    portion = {"B1": R, "B2": Y, "B3": B}
    color_volume = {"B1": R, "B2": Y, "B3": B}

    assert (
        p300 is not None
        and tiprack_1 is not None
        and reservoir is not None
        and plate is not None
    )

    for pos in position:
        if float(portion[pos]) != 0.0:
            p300.pick_up_tip(tiprack_1[pos])
            p300.aspirate(color_volume[pos], reservoir[pos])
            p300.dispense(color_volume[pos], plate[mix_well])
            p300.default_speed = 100
            p300.blow_out(reservoir["A1"].top(z=-5))
            p300.default_speed = 400
            p300.drop_tip(tiprack_1[pos])

    print(f"Mixed R:{R}, Y:{Y}, B:{B} in well {mix_well}")

@flow
def move_sensor_to_measurement_position(mix_well):
    assert p300 is not None and tiprack_2 is not None and plate is not None
    p300.pick_up_tip(tiprack_2["A2"])
    p300.move_to(plate[mix_well].top(z=-1.3))
    print("Sensor is now in position for measurement")

@flow
def move_sensor_back():
    assert p300 is not None and tiprack_2 is not None
    p300.drop_tip(tiprack_2["A2"].top(z=-80))
    print("Sensor moved back to charging position")

if __name__ == "__main__":

    serve(
        mix_color.to_deployment("mix-color"),
        move_sensor_to_measurement_position.to_deployment("move-sensor-to-measurement-position"),
        move_sensor_back.to_deployment("move-sensor-back"),
    )


Overwriting device.py


In [7]:
!python device.py

06:57:01.170 | WARNING | opentrons.config.robot_configs - /root/.opentrons/robot_settings.json not found. Loading defaults
06:57:01.172 | WARNING | opentrons.calibration_storage.ot2.deck_attitude - Deck calibration not found.
06:57:01.172 | WARNING | opentrons.config.robot_configs - /root/.opentrons/deck_calibration.json not found. Loading defaults
Labwares loaded
Your deployments are being served and polling for scheduled runs!

                                Deployments                                
┌─────────────────────────────────────────────────────────────────────────┐
│ mix-color/mix-color                                                     │
│ move-sensor-to-measurement-position/move-sensor-to-measurement-position │
│ move-sensor-back/move-sensor-back                                       │
└─────────────────────────────────────────────────────────────────────────┘

To trigger any of these deployments, use the following command:

        $ prefect deployment run [DEPLOYMENT